`.. _guide_monitor:`

# Monitor

[![Github](https://img.shields.io/github/stars/lab-ml/labml?style=social)](https://github.com/lab-ml/labml)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lab-ml/labml/blob/master/guides/monitor.ipynb)                    

`.. currentmodule:: labml.monit`

In [ ]:
%%capture
!pip install labml

## Iterators & Enumerators

You can use `:func:iterate` and `:func:enum` with any iterable object.
In this example we use a PyTorch `DataLoader`.

In [1]:
# Create a data loader for illustration
import time

import torch
from torchvision import datasets, transforms

from labml import logger, monit, lab, tracker

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(lab.get_data_path(),
                       train=False,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=32, shuffle=True)

In [2]:
for data, target in monit.iterate("Test", test_loader):
    time.sleep(0.01)

In [3]:
for i, (data, target) in monit.enum("Test", test_loader):
    time.sleep(0.01)

## Sections

Sections let you monitor time taken for different tasks and also helps keep the code clean by separating different blocks of code.

In [4]:
with monit.section("Load data"):
    # code to load data
    time.sleep(2)

In [5]:
with monit.section("Load saved model"):
    time.sleep(1)
    monit.fail()

You can also show progress while a section is running

In [6]:
with monit.section("Train", total_steps=100):
    for i in range(100):
        time.sleep(0.1)
        # Multiple training steps in the inner loop
        monit.progress(i)

## Loop

This can be used for the training loop. The :func:loop keeps track of the time taken and time remaining for the loop.

:func:labml.tracker.save outputs the current status along with global step.

In [7]:
for step in monit.loop(range(0, 400)):
    tracker.save()

In [8]:
tracker.set_global_step(0)

You can manually increment global step too.

In [9]:
for step in monit.loop(range(0, 400)):
    tracker.add_global_step(5)
    tracker.save()